# Resposta às perguntas do arquivo "perguntas_sql.md"

Este *python notebook* visa responder as questões 1 a 10 utilizando python, principalmente pandas. Considerando que a base principal tem mais de 10 milhões de linhas, farei duas *queries* usando o método `read_sql()` do pacote `basedosdados`.

In [ ]:
# Libs utilizadas
import pandas as pd
import numpy as np
import basedosdados as bd

# billing id para o basedosdados
bill_id = "teste-tecnico-pcrj"

# Funções que serão utilizadas ao longo do notebook

def soma_agrupada_chamados(df: pd.DataFrame,
                           colunas: list):
    """
    Função para somar chamados agrupando por colunas especificadas. Utilizada no conjunto de respostas 1-5.
    
    Args:
        df (pd.DataFrame): DataFrame com os dados.
        colunas (list): Lista com as colunas que serão utilizadas para agrupar os dados.
    
    Returns:
        pd.DataFrame: DataFrame com a soma dos chamados pelos grupos. A última coluna sempre será o total de chamados.
    """
    df_agrupado = (
        df.groupby(colunas)
        .agg({'total_chamados': 'sum'})
        .reset_index()
        .sort_values('total_chamados', ascending=False))
    
    return df_agrupado

## Respostas às perguntas 1-5

In [ ]:
# preparando objetos para as questões 1-5

sql_1 = """
SELECT 
  COUNT(id_chamado) AS total_chamados,
  DATE(data_inicio) AS data_abertura,
  tipo,
  subtipo,
  id_bairro
FROM `datario.adm_central_atendimento_1746.chamado` 
WHERE DATE(data_inicio)= '2023-04-01'
GROUP BY DATE(data_inicio), tipo, subtipo, id_bairro
"""

sql_2 = """
SELECT 
  id_bairro,
  nome AS nome_bairro,
  subprefeitura
FROM `datario.dados_mestres.bairro`
"""

df_base_1 = bd.read_sql(sql_1, billing_project_id=bill_id)
df_bairros = bd.read_sql(sql_2, billing_project_id=bill_id)

### 1. Quantos chamados foram abertos no dia 01/04/2023?

In [7]:
total_chamados_soma = df_base_1['total_chamados'].sum()

print(f'R: O total de chamados abertos no dia 01/04/2023 é de {total_chamados_soma} chamados.')

R: O total de chamados abertos no dia 01/04/2023 é de 1903 chamados.


### 2. Qual é o tipo de chamado que teve mais chamados abertos no dia 01/04/2023?

In [13]:
chamados_por_dia = soma_agrupada_chamados(df_base_1, ['tipo'])

print(f'R: O tipo de chamado com maior quantidade de chamados abertos no dia 01/04/2023 é {chamados_por_dia.iloc[0,0]} com {chamados_por_dia.iloc[0,1]} chamados.')

R: O tipo de chamado com maior quantidade de chamados abertos no dia 01/04/2023 é Estacionamento irregular com 373 chamados.


### 3. Quais os nomes dos bairros que mais tiveram chamados abertos nesse dia?

In [47]:
df_base_bairros = df_base_1.merge(df_bairros, on='id_bairro', how='left')

chamados_por_bairros = soma_agrupada_chamados(df_base_bairros, ['nome_bairro'])

top_n = 3

print(f'O top {top_n} bairros com maior quantidade de chamados abertos no dia 01/04/2023 foram:')
for i in range(top_n):
    print(f'{i+1} - {chamados_por_bairros.iloc[i,0]} com {chamados_por_bairros.iloc[i,1]} chamados.')

O top 3 bairros com maior quantidade de chamados abertos no dia 01/04/2023 foram:
1 - Campo Grande com 124 chamados.
2 - Tijuca com 96 chamados.
3 - Barra da Tijuca com 60 chamados.


### 4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [ ]:
df_base_bairros = df_base_1.merge(df_bairros, on='id_bairro', how='left')

chamados_por_subprefeitura = soma_agrupada_chamados(df_base_bairros, ['subprefeitura'])

print(f'A subprefeitura com maior quantidade de chamados abertos no dia 01/04/2023 foi a {chamados_por_subprefeitura.iloc[0,0]} com {chamados_por_subprefeitura.iloc[0,1]} chamados.')

A subprefeitura com maior quantidade de chamados abertos no dia 01/04/2023 foi a Zona Norte com 534 chamados.


### 5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece? 

In [31]:
df_bairro_nulo = df_base_1.loc[df_base_1['id_bairro'].isnull()]
df_bairro_nulo_por_tipo = soma_agrupada_chamados(df_bairro_nulo,
                                                 ['tipo'])

df_bairro_nulo_por_tipo.head(10)

,tipo,total_chamados
33,Ônibus,50
1,Atendimento ao cidadão,18
4,Clínicas da Família (CF),13
30,Sistema Nacional de Regulação (SISREG),6
21,Ouvidoria SMTR,5
14,Iluminação Pública,3
13,Hospitais (HM),3
8,Diversos - Comlurb,3
22,Postos de Saúde (PS),2
6,Conservação de vias,2


R: Sim, existiram chamados que não foram associados à nenhum id_bairro e, portanto, à nenhuma subprefeitura. Avaliando os principais tipos de chamados que possuem esta característica, é possível verificar que essas questões não dependem do espaço geográfico do cidadão que as origina. Por exemplo, chamados em relação aos ônibus não necessariamente serão questões de um só bairro, visto que existem ônibus (a grande maioria) cujo trajeto perpassa múltiplos bairros. Atendimento ao cidadão é outro caso, visto que dúvidas, sugestões, aviso de erros ou problemas com serviços digitais da prefeitura podem não ser ligados ao local em que o cidadão está fazendo o chamado. Ou seja, a maior parte dessas questões não são dependentes diretamente da localização.

## Respostas às perguntas 6-10

In [50]:
# Preparando os data frames básicos para as questões 6-10

sql_22_23 ="""
SELECT COUNT(id_chamado) AS total_chamados,
    DATE(data_inicio) AS data_abertura,
    subtipo
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
GROUP BY data_abertura, subtipo
"""

sql_eventos = """
SELECT
    evento,
    data_inicial,
    data_final
FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
WHERE data_inicial IS NOT NULL
"""

df_22_23 = bd.read_sql(sql_22_23, billing_project_id=bill_id)
df_eventos = bd.read_sql(sql_eventos, billing_project_id=bill_id)

Downloading: 100%|██████████|
Downloading: 100%|██████████|


### 6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [ ]:
subtipo_perturbacao_soma = (
    df_22_23.loc[df_22_23['subtipo'].str.contains('sossego', case=False, na=False),
                 'total_chamados'].sum()
)

subtipo_perturbacao_soma
df_22_23.loc[df_22_23['subtipo'].str.contains('sossego', case=False, na=False)].groupby('subtipo')['total_chamados'].sum().reset_index()

subtipo
Fiscalização de perturbação do sossego      50368
Informações sobre Perturbação do Sossego    11590
Name: total_chamados, dtype: Int64